# FAQ

## What is Steppy standard documentation framework?

The documentation framework is **Sphinx**

## What is the difference between scikit pipeline and steppy?

In a  pipeline a series of data-operations are connected together. In a data-analytics problem, this is usually a left-to-right series of calls of transformer calls ended by an estimator, that is a PIPELINE of invocations.

The major limitation the scikit learn Pipeline wrapper is that passed data object must be same input and output, i.e. data object(s) is(are) same and implicit thoughout Pipeline.

Thus step introduces two new wrappers, Step and Adapter to avoid these limitations:

Steps communicate data between each other with **Adapters**, which are implemented as Python dictionaries. This makes it possible to pass collections of arbitrary data types (Numpy arrays, Pandas dataframes, etc.). The basic structure is as follows:

    data_train = {'input':
                    {
                         'X': X_train,
                         'y': y_train,
                    }
                }

where X_train,y_train are local data objects, X,y are names of arguments to step-exec-object-instance and  ‘input'  is the name of the Adapter.


see https://github.com/neptune-ml/steppy-examples/blob/master/tutorials/1-getting-started.ipynb for more information.

## how to get around  xp =f(x)  to, x,xp = f(x) challenge

All of the argument/inputs of ALL step-exec-object-instance must be covered by the named Adapter, but all the named arguments of the named Adapter need not be used by the step-exec-object-instance. For example the following Adapter would work for a **Step** that needs only X .


    data_train = {'input':
                    {
            'X': X_train,
            'y': y_train,
            ‘z1’, some_other_bound_variable,
            ‘etc’, etc	
                    }
                }

## Are there other Resources for Steepy?

Yes. see  https://steppy.readthedocs.io

## Are there tutorials for Steepy?

Yes. In the form of notebooks. See https://github.com/neptune-ml/steppy-examples/tree/master/tutorials

## Is Steppy Threadsafe?

Steppy itself is  thread-safe. However, you will use Steppy with many other packages which may not be thread-safe. For example, numpy is thread-safe.
ndarrays can be accessed in a thread-safe manner, but you must be careful with state if you mutate an array.

In Pandas, deleting a column is usually not thread-safe as changing the size of a DataFrame usually results in a new Dataframe object. At some point this may change in Pandas and other python libaries as multi-cored CPUs are becoming common.

# 

# 

# 

# 

# 

# 